In [1]:
import sys
import os
import re
from docx import Document
from PyPDF2 import PdfReader
import gdown
from pptx import Presentation
sys.path.append('..')
import pandas as pd
from utils.configs import Config

In [2]:
#ROOT_DIR = Config.ROOT_DIR
ROOT_DIR = "D:\Study\Level4\grad project\VC-management-system"

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Yusuf Aldaly\AppData\Local\Temp\ipykernel_13632\605947850.py:2: SyntaxWarning: invalid escape sequence '\S'
  ROOT_DIR = "D:\Study\Level4\grad project\VC-management-system"


In [3]:
initial_data = pd.read_csv(os.path.join(ROOT_DIR, "data", "initial_data.csv"))
initial_data.head()

,id,email,gender,Nationality,place of current residence,residential status,company name,startup industry sector,idea overview,pitch deck link,number of full time and part time team members (including founders / co-founders),preferred language of instruction,founder/co-founder cv/linkedin link
0,1,founder1@example.com,Female,Egypt,Egypt,Citizen,GreenTech Solutions,Renewable Energy,Developing affordable solar panels for rural a...,C:\Users\Yusuf Aldaly\Desktop\AutoInspecta pit...,5,English,D:\college research& Applications\Graduate\You...
1,2,founder2@example.com,Male,Egypt,Egypt,Citizen,HealthBridge,HealthTech,Telemedicine platform connecting patients with...,http://example.com/pitch2.pdf,8,Arabic,D:\college research& Applications\Graduate\sam...
2,3,founder3@example.com,Prefer not to Answer,Egypt,Egypt,Citizen,AgriSmart,AgriTech,Smart irrigation systems for efficient water use,http://example.com/pitch3.pdf,4,English,http://linkedin.com/in/founder3
3,4,founder4@example.com,Female,Egypt,Egypt,Citizen,FinEdu,EdTech,Online financial literacy courses for youth,http://example.com/pitch4.pdf,6,Arabic,http://linkedin.com/in/founder4
4,5,founder5@example.com,Male,Egypt,Egypt,Citizen,ShopEase,E-Commerce,Marketplace for local artisans to sell handmad...,http://example.com/pitch5.pdf,7,English,http://linkedin.com/in/founder5


In [4]:
def detect_format(cv_entry):
    if isinstance(cv_entry, str):
        if re.search(r'linkedin\.com', cv_entry, re.IGNORECASE):
            return 'linkedin'
        elif re.search(r'drive\.google\.com', cv_entry, re.IGNORECASE):
            return 'google_drive'
        elif cv_entry.lower().endswith('.pdf'):
            return 'pdf'
        elif cv_entry.lower().endswith('.txt'):
            return 'txt'
        elif cv_entry.lower().endswith('.pptx'):
            return 'pptx'
        elif cv_entry.lower().endswith('.ppt'):
            return 'ppt'
        elif cv_entry.lower().endswith('.docx'):
            return 'docx'
        elif cv_entry.startswith('http'):
            return 'url'
        else:
            return 'unknown'
    return 'invalid'

In [5]:
def extract_docx_text(docx_path):
    doc = Document(docx_path)
    text = []

    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    
    return "\n".join(text)

In [6]:
def extract_pdf_text(pdf_path):
    reader = PdfReader(pdf_path)
    return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])


In [7]:
def extract_txt_text(txt_path):
    with open(txt_path, 'r', encoding='utf-8') as f:
        return f.read()


In [8]:
def download_drive_file(url, output_path='temp_download'):
    file_id = re.findall(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id:
        file_id = re.findall(r'id=([a-zA-Z0-9_-]+)', url)
    if file_id:
        gdown.download(f"https://drive.google.com/uc?id={file_id[0]}", output_path, quiet=False)
        return output_path
    return None


In [9]:
def extract_linkedin_summary(link):
    return f"LinkedIn Profile: {link}" 


In [10]:
def extract_pptx_text(pptx_path):
    prs = Presentation(pptx_path)
    text = []

    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text.append(shape.text)
    
    return "\n".join(text)


In [11]:
def extract_doc_text(cv_entry):
    fmt = detect_format(cv_entry)

    try:
        if fmt == 'pdf':
            return extract_pdf_text(cv_entry)
        elif fmt == 'txt':
            return extract_txt_text(cv_entry)
        elif fmt == 'pptx':
            return extract_pptx_text(cv_entry)
        elif fmt == 'docx':
            return extract_docx_text(cv_entry)
        elif fmt == 'google_drive':
            file_path = download_drive_file(cv_entry)
            if file_path.endswith('.pdf'):
                return extract_pdf_text(file_path)
            elif file_path.endswith('.txt'):
                return extract_txt_text(file_path)
            elif file_path.endswith('.docx'):
                return extract_docx_text(file_path)
        elif fmt == 'linkedin':
            return extract_linkedin_summary(cv_entry)
        elif fmt == 'url':
            return f"Generic URL: {cv_entry}"
        else:
            return "Unsupported or invalid CV format."
    except Exception as e:
        return f"Error extracting: {str(e)}"


In [12]:
initial_data['pith_deck_text'] = initial_data['pitch deck link'].apply(extract_doc_text)
initial_data.head()

,id,email,gender,Nationality,place of current residence,residential status,company name,startup industry sector,idea overview,pitch deck link,number of full time and part time team members (including founders / co-founders),preferred language of instruction,founder/co-founder cv/linkedin link,pith_deck_text
0,1,founder1@example.com,Female,Egypt,Egypt,Citizen,GreenTech Solutions,Renewable Energy,Developing affordable solar panels for rural a...,C:\Users\Yusuf Aldaly\Desktop\AutoInspecta pit...,5,English,D:\college research& Applications\Graduate\You...,\nSyncPert\n\nManuTech 2023\n\n\nPROBLEM\n\n\n...
1,2,founder2@example.com,Male,Egypt,Egypt,Citizen,HealthBridge,HealthTech,Telemedicine platform connecting patients with...,http://example.com/pitch2.pdf,8,Arabic,D:\college research& Applications\Graduate\sam...,Error extracting: [Errno 22] Invalid argument:...
2,3,founder3@example.com,Prefer not to Answer,Egypt,Egypt,Citizen,AgriSmart,AgriTech,Smart irrigation systems for efficient water use,http://example.com/pitch3.pdf,4,English,http://linkedin.com/in/founder3,Error extracting: [Errno 22] Invalid argument:...
3,4,founder4@example.com,Female,Egypt,Egypt,Citizen,FinEdu,EdTech,Online financial literacy courses for youth,http://example.com/pitch4.pdf,6,Arabic,http://linkedin.com/in/founder4,Error extracting: [Errno 22] Invalid argument:...
4,5,founder5@example.com,Male,Egypt,Egypt,Citizen,ShopEase,E-Commerce,Marketplace for local artisans to sell handmad...,http://example.com/pitch5.pdf,7,English,http://linkedin.com/in/founder5,Error extracting: [Errno 22] Invalid argument:...


In [13]:
pith_deck_data = initial_data['pith_deck_text'].iloc[0]
pith_deck_data

'\nSyncPert\n\nManuTech 2023\n\n\nPROBLEM\n\n\n\nLack of Task Prioritization\nInefficient Random Routes\nLimited Traceability\nPROBLEM\n\n\n\n\n\n\n\n\n\n\n48 DAYS \nAre wasted each year on unnecessary motion\n$4.3 BILLION\nIn costs for just the warehouse industry\n15% EACH DAY\nwasted on non-productive activities (no automated tracking systems)\n265 MILLION\nOf labor hours are wasted annually\nPROBLEM\n\n\n\n\n\n\nMARKET OVERVIEW\nROUTE OPTIMIZATION \nLOGISTICS\nMARKET\nIN EGYPT BY 2029\n  WITH AN UPWARD TREND\n$12.4 Billion\n$18 Billion\n28% Growth\nGLOBAL MARKET BY 2030\nGLOBAL LOGISTICS \n\n\n\n\n\n\nTHE SOLUTION\nSmart Task Prioritization\nCustomized Route Optimization\nReal-Time Line Runner Traceability\n01\n02\n03\nA seamless restocking solution that ensures no delays while saving time and money\nPRODUCT DEMO\n\n\n\n\n\n\nLINE RUNNER INTERFACE\nPRODUCT DEMO\n\n\n\n\n\n\n\n\n\n\nEvery manufacturing facility with kanban system\nManufacturing facilities with kanban system in MENA R

In [14]:
initial_data['cv_text'] = initial_data['founder/co-founder cv/linkedin link'].apply(extract_doc_text)
initial_data.head()

,id,email,gender,Nationality,place of current residence,residential status,company name,startup industry sector,idea overview,pitch deck link,number of full time and part time team members (including founders / co-founders),preferred language of instruction,founder/co-founder cv/linkedin link,pith_deck_text,cv_text
0,1,founder1@example.com,Female,Egypt,Egypt,Citizen,GreenTech Solutions,Renewable Energy,Developing affordable solar panels for rural a...,C:\Users\Yusuf Aldaly\Desktop\AutoInspecta pit...,5,English,D:\college research& Applications\Graduate\You...,\nSyncPert\n\nManuTech 2023\n\n\nPROBLEM\n\n\n...,Yousef M. Eldaly \n20220527@stud.fci -cu.edu....
1,2,founder2@example.com,Male,Egypt,Egypt,Citizen,HealthBridge,HealthTech,Telemedicine platform connecting patients with...,http://example.com/pitch2.pdf,8,Arabic,D:\college research& Applications\Graduate\sam...,Error extracting: [Errno 22] Invalid argument:...,"Yousef M. Eldaly\n\n12 Agyad Mecca • Haram, 12..."
2,3,founder3@example.com,Prefer not to Answer,Egypt,Egypt,Citizen,AgriSmart,AgriTech,Smart irrigation systems for efficient water use,http://example.com/pitch3.pdf,4,English,http://linkedin.com/in/founder3,Error extracting: [Errno 22] Invalid argument:...,LinkedIn Profile: http://linkedin.com/in/founder3
3,4,founder4@example.com,Female,Egypt,Egypt,Citizen,FinEdu,EdTech,Online financial literacy courses for youth,http://example.com/pitch4.pdf,6,Arabic,http://linkedin.com/in/founder4,Error extracting: [Errno 22] Invalid argument:...,LinkedIn Profile: http://linkedin.com/in/founder4
4,5,founder5@example.com,Male,Egypt,Egypt,Citizen,ShopEase,E-Commerce,Marketplace for local artisans to sell handmad...,http://example.com/pitch5.pdf,7,English,http://linkedin.com/in/founder5,Error extracting: [Errno 22] Invalid argument:...,LinkedIn Profile: http://linkedin.com/in/founder5
